In [1]:
import requests
import json
import pprint
import pandas as pd

In [2]:
congress_api_key = "hi8Fnm4tMO7CV7lgUfvOqJVkPBnORYoDpqVdnmlE"

In [3]:
limit = 950
offset = 751

In [6]:
csv_path = "C:/Users/17202/Documents/GitHub/project1/legislator_info.csv"

legislator_info_df = pd.read_csv(csv_path)


In [7]:
legislator_info_df

,Unnamed: 0,Bio Guide ID,Name,Party,State,API URL,District Number,Senate End Year,Senate Start Year,House Start Year,House End Year
0,0,S000062,"Sarasin, Ronald A.",Republican,Connecticut,https://api.congress.gov/v3/member/S000062?for...,5.0,NaN,NaN,1973.0,1979.0
1,1,B001305,"Budd, Ted",Republican,North Carolina,https://api.congress.gov/v3/member/B001305?for...,NaN,NaN,2023.0,2017.0,2023.0
2,2,W000817,"Warren, Elizabeth",Democratic,Massachusetts,https://api.congress.gov/v3/member/W000817?for...,NaN,NaN,2013.0,NaN,NaN
3,3,W000802,"Whitehouse, Sheldon",Democratic,Rhode Island,https://api.congress.gov/v3/member/W000802?for...,NaN,NaN,2007.0,NaN,NaN
4,4,Y000064,"Young, Todd",Republican,Indiana,https://api.congress.gov/v3/member/Y000064?for...,NaN,NaN,2017.0,2011.0,2017.0
...,...,...,...,...,...,...,...,...,...,...,...
2495,2495,F000349,"Franks, Bob",Republican,New Jersey,https://api.congress.gov/v3/member/F000349?for...,7.0,NaN,NaN,1993.0,2001.0
2496,2496,F000347,"Franklin, William W.",Republican,Mississippi,https://api.congress.gov/v3/member/F000347?for...,2.0,NaN,NaN,1983.0,1987.0
2497,2497,F000339,"Frank, Barney",Democratic,Massachusetts,https://api.congress.gov/v3/member/F000339?for...,4.0,NaN,NaN,1981.0,2013.0
2498,2498,F000332,"Fox, Jon D.",Republican,Pennsylvania,https://api.congress.gov/v3/member/F000332?for...,13.0,NaN,NaN,1995.0,1999.0


In [ ]:
url = f"https://api.congress.gov/v3/member?limit={limit}&offset={offset}&api_key={congress_api_key}"

response = requests.get(url).json()
# pprint.pprint(response)

In [ ]:
members_num = len(response["members"])
members=[]

In [ ]:
members_num 

In [ ]:
for i in range(members_num):
    ID = response["members"][i]["member"]["bioguideId"]
    members.append(ID)
# print(members)

In [ ]:
gender = []

for x in members: 
    url = f"https://api.congress.gov/v3/member/{x}?limit={limit}&offset={offset}&api_key={congress_api_key}"

    response = requests.get(url).json()
    member = response["member"]
    name = member["invertedOrderName"]

    honorific_name = member["honorificName"] if "honorificName" in member else None

    if honorific_name:
        if honorific_name == "Mr.":
            sex = "M"
        elif honorific_name in ("Mrs.", "Ms.", "Miss"):
            sex = "F"
        else:
            sex = "U"
    else:
        sex = "U"

    gender.append({"Name" :name,
                   "Gender": sex})

In [ ]:
print(gender)

In [ ]:
congress_gender_df = pd.DataFrame(gender)

In [ ]:
congress_gender_df

In [ ]:
merged_df = pd.merge(legislator_info_df, congress_gender_df, on = "Name" )

In [ ]:
merged_df

In [ ]:
bill_member =[]

for z in members:
    url = f"https://api.congress.gov/v3/member/{z}?limit={limit}&offset={offset}&api_key={congress_api_key}"
    response = requests.get(url).json()
    member = response["member"]
    name = member["invertedOrderName"]
    
    bills = member["sponsoredLegislation"]["count"] if "sponsoredLegislation" in member else None
    
    bill_member.append({"Name" : name,
                       "Bill" : bills})

In [ ]:
print(bill_member)

In [ ]:
bill_congress = pd.DataFrame(bill_member)

bill_congress

In [ ]:
combined_df = pd.merge(merged_df, bill_congress, on="Name")

In [ ]:
combined_df

In [ ]:
combined_df.to_csv("Congress_Gender_1000.csv", index=False)